In [40]:
import pandas as pd
import os, json, re, datetime

bigbench_task_folder = "/home/junetheriver/codes/qa_generation/BIG-bench/bigbench/benchmark_tasks"

tasks = []

for task_folder in os.listdir(bigbench_task_folder):
    task_file = os.path.join(bigbench_task_folder, task_folder, "task.json")
    if not os.path.exists(task_file):
        continue
    with open(task_file, "r") as f:
        task = json.load(f)

    task_json = {}

    concern_fields = ['name', 'description', 'keywords', 'metrics', 'examples']
    for field in concern_fields:
        if field in task:
            if field == 'examples':
                task_json[field] = [task[field][0]]
            else:
                task_json[field] = task[field]
    
    tasks.append(task_json)

In [6]:
len(tasks)

172

In [23]:
current_abilities_examples_file = "/home/junetheriver/codes/qa_generation/huawei/dogeneval/prompts/examples.json"

abilities_examples = json.load(open(current_abilities_examples_file, "r"))

for cat in abilities_examples:
    abilities_examples[cat] = abilities_examples[cat][0]
    

len(json.dumps(abilities_examples, ensure_ascii=False, indent=4))

28601

In [20]:
from dogeneval.template.abilities import ability_templates

for ability in ability_templates:
    del ability['examples']

len(json.dumps(ability_templates, ensure_ascii=False, indent=4))


5249

In [27]:
from dogeneval.utils.llm.llms import get_azure_model, get_openai_model
from dogeneval.utils.token_utils import get_qwen_token_num
from loguru import logger

llm = get_openai_model()
logger.info(llm.chat("测试，请回复API test success"))

ability_template_text = json.dumps(ability_templates, ensure_ascii=False, indent=4)
ability_template_text_token_num = get_qwen_token_num(ability_template_text)
logger.info(f"ability_template_text_token_num: {ability_template_text_token_num}")

2024-09-14 16:15:20.995 | INFO     | __main__:<module>:6 - API test success
2024-09-14 16:15:21.006 | INFO     | __main__:<module>:10 - ability_template_text_token_num: 1758


In [34]:
PROMPT = """你是华为大模型的评测集构建专家，需要构建一个通信领域的大模型评测体系，现在基于产品文档设计了以下任务，它们的描述、模板如下：

=============================================
{ability_template_text}
=============================================

现在对于BIG-Bench中的这个任务，请你判断如下几点：
1. 这个任务是否已经属于已有任务中的某一个或属于其变种、子集
2. 这个任务是否符合通信领域大模型评测的范畴

这个任务的描述是：
=============================================
{task_description}
=============================================

如果你认为这个任务不适合加入到现有任务中，返回一个json，格式如下：

{{
    "task_name": "任务名称",
    "decision": "reject",
    "reason": "原因"
}}

如果你认为这个任务适合加入到现有任务中，你需要为这个任务设计一个模板，并描述构建此类任务的题目应该采取的步骤或流程，返回一个json，格式如下：

{{
    "task_name": "任务名称（英文）",
    "decision": "accept",
    "reason": "原因",
    "ability": "总结任务的上层抽象能力，如：逻辑推理",
    "task": "翻译成中文的任务描述",
    "template": "题目模板",
    "fields": [
        {{
            "name": "字段名",
            "type": "字段类型",
            "description": "字段描述"
        }}
    ],
    "steps": "题目构建步骤或流程"
}}

你返回的内容需要是纯json，开头不要带```json，结尾不要带```
"""

In [44]:
from tqdm import tqdm
from dogeneval.utils.parser import try_parse_json

results = []

for task in tqdm(tasks):
    task_description = json.dumps(task, ensure_ascii=False, indent=4)
    prompt = PROMPT.format(ability_template_text=ability_template_text, task_description=task_description)
    
    token_len = get_qwen_token_num(prompt)
    tqdm.write(f"token_len: {token_len}")
    
    if token_len > (32768 - 4096):
        tqdm.write(f"任务 {task['name']} 的token数超过限制，跳过")
        tqdm.write(prompt)
        continue

    response = llm.chat(prompt)

    response_json = try_parse_json(response)

    if response_json:
        if response_json['decision'] == 'accept':
            tqdm.write(f"任务 {task['name']} 被接受")
            tqdm.write(f"response_json: {response_json}")
        else:
            tqdm.write(f"任务 {task['name']} 被拒绝，原因：{response_json['reason']}")

        response_json['original_task'] = task
        results.append(response_json)
    else:
        tqdm.write(f"任务 {task['name']} 的判断失败")
        results.append({
            "task_name": task['name'],
            "decision": "fail",
            "reason": "判断失败",
            "original_task": task
        })
    
    # 临时保存

    tmp_file = "/home/junetheriver/codes/qa_generation/huawei/data/bigbench_mapping_results.jsonl"
    with open(tmp_file, "at+") as f:
        f.write(json.dumps(results[-1], ensure_ascii=False))
        f.write("\n")

# 保存结果
with open("/home/junetheriver/codes/qa_generation/huawei/data/bigbench_mapping_results.json", "w") as f:
    f.write(json.dumps(results, ensure_ascii=False, indent=4))

  0%|          | 0/172 [00:00<?, ?it/s]

token_len: 2106


  1%|          | 1/172 [00:07<22:24,  7.86s/it]

任务 abstract_narrative_understanding 被接受
response_json: {'task_name': 'abstract_narrative_understanding', 'decision': 'accept', 'reason': '此任务虽然不直接属于通信领域，但其涉及的社会推理、叙事理解和类比推理在通信领域的故障排查、用户行为预测等场景中有潜在的应用价值。', 'category': '语义理解', 'template': '给定一段叙述性文本和一系列谚语，你需要选择最能反映叙述文本核心思想的谚语。叙述文本：{narrative_text}，谚语列表：{proverb_list}', 'fields': [{'name': 'narrative_text', 'type': 'string', 'description': '叙述性文本'}, {'name': 'proverb_list', 'type': 'list', 'description': '谚语列表'}], 'steps': '首先，理解叙述性文本的核心意义；其次，对比谚语列表中每个谚语的意义；最后，选择最能反映叙述文本核心思想的谚语。'}
token_len: 2184


  1%|          | 2/172 [00:11<15:48,  5.58s/it]

...

任务 cifar10_classification 被拒绝，原因：此任务涉及图像分类，属于计算机视觉领域，与提供的通信领域大模型评测任务不相符，且不属于语言处理或通信技术相关的任务类型
token_len: 2213


 10%|█         | 18/172 [01:49<16:10,  6.30s/it]

任务 code_line_description 被接受
response_json: {'task_name': 'code_line_description', 'decision': 'accept', 'reason': '这个任务属于代码理解和描述的范畴，与通信领域的代码MML生成任务有相似之处，可以视为代码理解的一个子集，适合加入到现有任务中。', 'category': '代码MML生成', 'template': '给定一段Python代码，你需要用英文描述这段代码的功能和执行逻辑。你的回答应详细说明代码的每一行做了什么，以及整体代码的运行效果。', 'fields': [{'name': 'code_snippet', 'type': 'string', 'description': '需要描述的Python代码片段'}], 'steps': '构建此类任务时，首先需要准备一段Python代码作为输入，然后设计多个选项供模型选择，其中只有一个正确答案描述了代码的真实功能。'}
token_len: 2223


 10%|█         | 18/172 [01:52<15:58,  6.22s/it]


KeyboardInterrupt: 